In [2]:
import os
import pickle
import sparse
# import librosa
import mir_eval
import fluidsynth
import pretty_midi
import numpy as np
import IPython.display

from glob import glob
from utils.drum_utils import *
from utils.common_utils import *

%load_ext autoreload
%autoreload 2

### Preprocessing

In [13]:
data = []
count = 0 # num of bars
data_path = '../drumVAE/data/drummer1/session1'
save_path = 'midi_data.pkl'

In [23]:
for file_path in glob(data_path+'/*'):
#     try:
    pm = pretty_midi.PrettyMIDI(file_path)

    # time signature 4/4 check
    if not check_time_sign(pm, num=4, denom=4): 
        continue

    start_time = pm.get_onsets()[0]
    beats = pm.get_beats(start_time) #beat location in second
    tempo = pm.estimate_tempo()#bpm
    fs = change_fs(beats)

    # for each inst
    for inst in pm.instruments:
#         print(inst.notes)
        if inst.is_drum == True:
            drum_roll = quantize_drum(inst, fs, start_time)
            drum_roll = windowing(drum_roll) #64 / 16 = 4 measure
            drum_roll = hot_encoding(drum_roll)

            for i in range(0, drum_roll.shape[0]):
                # to reduce size of data, sparse encoding
                data.append(sparse.COO(drum_roll[i]))
#     except:
#         continue

    # verbose
    count += 1
    if count % 100 == 0:
        print('Files iterations %d' % count)

[Note(start=1.532892, end=1.634208, pitch=44, velocity=22), Note(start=1.715787, end=1.817103, pitch=38, velocity=84), Note(start=1.878945, end=1.980260, pitch=40, velocity=115), Note(start=2.163155, end=2.264470, pitch=44, velocity=87), Note(start=2.338154, end=2.380260, pitch=38, velocity=20), Note(start=2.380260, end=2.415786, pitch=38, velocity=23), Note(start=2.415786, end=2.439470, pitch=38, velocity=16), Note(start=2.498680, end=2.598680, pitch=36, velocity=59), Note(start=2.505259, end=2.607891, pitch=22, velocity=91), Note(start=2.644733, end=2.746049, pitch=38, velocity=21), Note(start=2.807891, end=2.909206, pitch=42, velocity=38), Note(start=2.815785, end=2.917101, pitch=38, velocity=22), Note(start=2.969732, end=3.071048, pitch=36, velocity=50), Note(start=2.969732, end=3.071048, pitch=42, velocity=20), Note(start=3.123680, end=3.223679, pitch=22, velocity=84), Note(start=3.298679, end=3.399995, pitch=40, velocity=102), Note(start=3.448679, end=3.549995, pitch=42, velocity

In [15]:
len(data)

0

In [12]:
# play music example
fs = 5
pm = drum_play(data[0].todense().astype('float32'), fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)

IndexError: list index out of range

In [50]:
# save pickle
with open(save_path, 'wb') as f:
    print('File saved!')
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# laod pickle
with open(save_path, 'rb') as f:
    a = pickle.load(f)

File saved!
